In [1]:
# License: Apache 2.0
!wget https://ia800700.us.archive.org/6/items/ga.ie.cll.48000.tar/ga.ie.cll.48000.tar.gz -O data/irish/ga.ie.cll.48000.tar.gz
!wget https://raw.githubusercontent.com/Idlak/Living-Audio-Dataset/master/ga/text.xml -O data/irish/ga.ie.cll.xml

--2021-03-26 00:44:46--  https://ia800700.us.archive.org/6/items/ga.ie.cll.48000.tar/ga.ie.cll.48000.tar.gz
Resolving ia800700.us.archive.org (ia800700.us.archive.org)... 207.241.230.70
Connecting to ia800700.us.archive.org (ia800700.us.archive.org)|207.241.230.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 286318482 (273M) [application/octet-stream]
Saving to: ‘data/irish/ga.ie.cll.48000.tar.gz’

data/irish/ga.ie.cl 100%[===================>] 273.05M   304KB/s    in 9m 49s  

2021-03-26 00:54:36 (474 KB/s) - ‘data/irish/ga.ie.cll.48000.tar.gz’ saved [286318482/286318482]

--2021-03-26 00:54:36--  https://raw.githubusercontent.com/Idlak/Living-Audio-Dataset/master/ga/text.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10993

In [ ]:
!tar zxvf data/irish/ga.ie.cll.48000.tar.gz -C /workspace/data/irish/

In [5]:
!pip install bs4
!pip install lxml

In [18]:
from bs4 import BeautifulSoup
import unicodedata
soup = BeautifulSoup(open('/workspace/data/irish/ga.ie.cll.xml').read(), 'lxml')
dataset = list()
for entry in soup.find_all('fileid'):
    current = dict()
    current['id'] = entry['id']
    current['text'] = unicodedata.normalize('NFC', entry.text.strip())
    dataset.append(current)

In [30]:
def is_upper_vowel(letter):
  if letter in ['A', 'E', 'I', 'O', 'U', 'Á', 'É', 'Í', 'Ó', 'Ú']:
    return True
  else:
    return False

def irish_lower(word):
  if len(word) > 1 and word[0] in ['n', 't'] and is_upper_vowel(word[1]):
    return word[0] + '-' + word[1:].lower()
  else:
    return word.lower()

def irish_lower_sentence(sentence):
  return " ".join([irish_lower(w) for w in sentence.split(" ")])

In [31]:
import re
hyphens = 'cll_z0001_713 cll_z0001_804 cll_z0002_069 cll_z0002_296 cll_z0002_448 cll_z0002_481 cll_z0002_484 cll_z0002_495'.split(' ')
for entry in dataset:
    tmp = entry['text']
    tmp = re.sub(' \- ', ' ', tmp)
    tmp = re.sub(' – ', ' ', tmp)
    tmp = re.sub('[‘“”\"\.\?!,–—;:]', '', tmp)
    if entry['id'] in hyphens:
        tmp = re.sub('\'', '', tmp)
    entry['sentence'] = irish_lower_sentence(tmp)

In [29]:
for entry in dataset:
    entry['speaker'] = 'caoimhin'
    entry['accent'] = 'dublin'
    entry['gender'] = 'male'
    entry['path'] = '/workspace/data/irish/48000_orig/{}.wav'.format(entry['id'])

In [34]:
import json
datasetjson = json.dumps(dataset)
jsonf = open("/workspace/data/irish/ga.ie.cll.json", "w")
jsonf.write(datasetjson)
jsonf.close()

In [2]:
import json
with open("/workspace/data/irish/ga.ie.cll.json") as jsonf:
    data = json.load(jsonf)

In [3]:
paths = [e['path'] for e in data]
sentences = [e['sentence'] for e in data]
dictds = {'path': paths,
          'sentence': sentences}

In [6]:
from datasets import Dataset
tmpds = Dataset.from_dict(dictds)
tmpds.save_to_disk('/workspace/data/irish/ga.ie.cll')

In [7]:
from datasets import Dataset
livingaudio = Dataset.load_from_disk('/workspace/data/irish/ga.ie.cll')